In [2]:
import os
import glob
import cv2
from PIL import Image
import numpy as np
from openpyxl import Workbook
from openpyxl.drawing.image import Image as Image_openpyxl
from openpyxl.utils import get_column_letter
from openpyxl.styles import Alignment
from natsort import natsorted

In [3]:
gt_dir = "/Users/imrankabir/Desktop/research/vqa_accessibility/Dashboard-For-VQA/Dashboard Data/GT_N"
images_dir = '/Users/imrankabir/Desktop/research/vqa_accessibility/Dashboard-For-VQA/Dashboard Data/Images'

In [4]:
def create_collage(list_of_images, width=1000, height=563):
    cols = 4
    rows = (len(list_of_images) // 4) + 1
    width = width
    height = int((rows / cols) * height)
    thumbnail_width = width//cols
    thumbnail_height = height//rows
    size = thumbnail_width, thumbnail_height
    new_im = Image.new('RGB', (width, height))
    ims = []
    for p in list_of_images:
        im = Image.open(p)
        im.thumbnail(size)
        ims.append(im)
    i = 0
    x = 0
    y = 0
    b_l = False
    for row in range(rows):
        for col in range(cols):
            if i >= len(list_of_images):
                b_l = True
                break
            # print(y, x)
            new_im.paste(ims[i], (x, y))
            i += 1
            if row % 2 == 0:  
                x += thumbnail_width + 10
            else:
                x -= thumbnail_width + 10
        y += thumbnail_height
        if row % 2 == 0:  
            x -= thumbnail_width + 10
        else:
            x += thumbnail_width + 10
        if b_l:
            break

    return new_im

In [5]:
def create_image_collage(img_dir, f_n, width=1000, height=563):
    images_path = natsorted(glob.glob(
        os.path.join(
            img_dir,
            f'{f_n}*'
        )
    ))

    new_im = create_collage(images_path, width, height)
    return new_im

In [6]:
pth = "/Users/imrankabir/Desktop/corr_plots_vtrbi"

all_corr_plot = create_image_collage(pth, "", width=2150, height=1580)

In [7]:
all_corr_plot.save("corr_plots_vtrbi_2.png")

In [99]:
gts = natsorted(os.listdir(gt_dir))

skip_list = [
    "video-10-segment-1.csv", "video-9-segment-2.csv"
]

gts = [x.split('.')[0] for x in gts if x.endswith('.csv') and int(x.split('-')[1])<=16 and x not in skip_list]

In [100]:
workbook = Workbook()
worksheet = workbook.active
out_plot_path = './col_image'

r = 2
c = 1
worksheet.row_dimensions[r].height = 1950
for gt in gts:
    v_ = int(gt.split('-')[1])
    s_ = int(gt.split('-')[3])

    worksheet.cell(row=1, column=c).value = f'Video {v_} Segment {s_}'
    worksheet.cell(row=1, column=c).alignment = Alignment(horizontal='center', vertical='center')
        
    collage_img = create_image_collage(images_dir, gt)

    img_out_name = os.path.join(out_plot_path, f'v{v_}s{s_}.png')

    collage_img.save(
        img_out_name
    )

    worksheet.column_dimensions[get_column_letter(c)].width = 128

    anchor = f'{get_column_letter(c)}{r}'
    # worksheet.add_image(Image_openpyxl(img_out_name), anchor=anchor)
    img = Image_openpyxl(img_out_name) 

    worksheet.add_image(img, anchor=anchor)

    c = c + 1

workbook.save('image_report.xlsx')